### Importy

In [34]:
import json
import string
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize
import morfeusz2
import itertools as it
import operator as op


1. Utworzenie korpusu dokumentów

In [ ]:
corpus_dir = "./literature" 
corpus = PlaintextCorpusReader(corpus_dir, ".*\.txt")
files = corpus.fileids()
files

2.  Wstępne przetworzzenie dokumentów

In [ ]:
documents = {}
for f in files:
    documents[f] = corpus.raw(f)
print(json.dumps(documents, indent=4, ensure_ascii=False))


In [ ]:
stoplist_file = open('stopwords_pl.txt', 'r', encoding='UTF-8')
stoplist = stoplist_file.read().splitlines()
stoplist_file.close()
stoplist = stoplist[4:]
stoplist

In [38]:
def lemmatize(text):
    morf = morfeusz2.Morfeusz()
    segments = it.groupby(morf.analyse(text), op.itemgetter(0,1))

    def disambiguate(group):
        pairs = ((len(descr), lemma)
                for _, _, (_, lemma, descr, _, _,) in group)
        perpl, lemma = min(pairs)
        return lemma.split(":")[0]
    
    lemmas = (disambiguate(group) for key, group in segments)
    return " ".join(filter(str.isalpha, lemmas))


In [ ]:
for key in documents:
    documents[key] = documents[key].lower()
    documents[key] = "".join([char for char in documents[key] if char not in string.punctuation])
    documents[key] = lemmatize(documents[key])
    documents[key] = " ".join([word for word in word_tokenize(documents[key], language='polish') if word not in stoplist])
print(json.dumps(documents, indent=4, ensure_ascii=False))